In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests
from json import load
import jsonschema
from jsonschema import validate
import json


# Open JSON file

In [3]:
# schema_file = "Agri_schema.json"
# schema_df = pd.read_json(schema_file)
# schema_df.head()

In [4]:
# with open('Agri_schema.json') as sc:
#     schema = load(sc)

# print(schema)

In [5]:
with open('Agri_schema.json', "r") as sc:
    schema = load(sc)
    
    pretty_json = json.dumps(schema, indent=3)
    print(pretty_json)

{
   "title": "agri-environmental-indicators-emissions-by-sector",
   "type": "object",
   "oneOf": [
      {
         "$ref": "#/definitions/agri-environmental-indicators-emissions-by-sector"
      }
   ],
   "definitions": {
      "agri-environmental-indicators-emissions-by-sector": {
         "properties": {
            "records": {
               "type": "array",
               "items": {
                  "$ref": "#/definitions/agri-environmental-indicators-emissions-by-sector_records"
               }
            }
         }
      },
      "agri-environmental-indicators-emissions-by-sector_records": {
         "properties": {
            "fields": {
               "type": "object",
               "properties": {
                  "year": {
                     "type": "string",
                     "format": "date",
                     "title": "Year",
                     "description": ""
                  },
                  "area": {
                     "type": "string",


In [6]:
json_file ="agri-environmental-indicators-emissions-by-sector.json"

In [7]:
entire_file_df = pd.read_json(json_file)
entire_file_df.head()

,datasetid,recordid,fields,record_timestamp
0,agri-environmental-indicators-emissions-by-sector,5500592b928d56fc0da60501fa364fa5c132b79a,"{'year': '2014', 'element': 'Emissions (F-gase...",2021-11-07T05:29:57.595-06:00
1,agri-environmental-indicators-emissions-by-sector,6b85b6ddf5ef6956602f5794450b16671561f5ff,"{'year': '2018', 'element': 'Emissions (F-gase...",2021-11-07T05:29:57.595-06:00
2,agri-environmental-indicators-emissions-by-sector,7315471f71541c41f2a5349f7bb4b522a051931f,"{'year': '1991', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00
3,agri-environmental-indicators-emissions-by-sector,825a93245b1d222469a12604a1651cc34a63bd52,"{'year': '1999', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00
4,agri-environmental-indicators-emissions-by-sector,3f4119c9755d2fddac4126a472f149131a6bace1,"{'year': '2012', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00


In [8]:
# with open(json_file, "r") as df:
#     aggschema = load(df)
    
#     pretty_json2 = json.dumps(aggschema, indent=3)
#     print(pretty_json2)

### IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

In [9]:
fields_df = entire_file_df.loc[:, "fields"]
fields_df.head()

0    {'year': '2014', 'element': 'Emissions (F-gase...
1    {'year': '2018', 'element': 'Emissions (F-gase...
2    {'year': '1991', 'element': 'Emissions Share (...
3    {'year': '1999', 'element': 'Emissions Share (...
4    {'year': '2012', 'element': 'Emissions Share (...
Name: fields, dtype: object

In [32]:
ent_json = pd.json_normalize(fields_df)
ent_json.head()

,year,element,unit,area,item,value
0,2014,Emissions (F-gases),kilotonnes,Mongolia,Waste - food systems,0.0000
1,2018,Emissions (F-gases),kilotonnes,Mongolia,Waste - food systems,0.0000
2,1991,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.9332
3,1999,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.3126
4,2012,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.6152


In [27]:
# fields_df.describe() - bad idea


In [28]:
# new_fields_df = pd.DataFrame.from_dict(fields_df)
# new_fields_df